In [2]:
import os
from tabnanny import verbose

def rename_files_and_folders(root_path):
    for dirpath, dirnames, filenames in os.walk(root_path):
        # Renommer les dossiers
        for dirname in dirnames:
            new_dirname = dirname.encode("ascii", "ignore").decode("ascii").replace(" ", "_")
            os.rename(os.path.join(dirpath, dirname), os.path.join(dirpath, new_dirname))

        # Renommer les fichiers
        for filename in filenames:
            new_filename = filename.encode("ascii", "ignore").decode("ascii").replace(" ", "_")
            os.rename(os.path.join(dirpath, filename), os.path.join(dirpath, new_filename))

dataset_path = "dataset(test)"
rename_files_and_folders(dataset_path)
print("Renommage terminé.")


Renommage terminé.


In [1]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
from deepface import DeepFace
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
import os
import cv2
import csv
import tensorflow as tf
import pickle
import threading
from datetime import datetime
from ultralytics import YOLO
import logging
import sys
import json
import hashlib

class LoginWindow:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Connexion")
        self.root.geometry("300x250")  # Augmenté la hauteur pour le nouveau menu

        # Variables
        self.password_var = tk.StringVar()
        self.action_var = tk.StringVar(value="Entrée")
        self.salle_var = tk.StringVar(value="100")  # Valeur par défaut pour la salle

        self.create_widgets()

    def create_widgets(self):
        main_frame = ttk.Frame(self.root, padding="20")
        main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

        # Password field
        ttk.Label(main_frame, text="Mot de passe:").grid(row=0, column=0, pady=10)
        password_entry = ttk.Entry(main_frame, textvariable=self.password_var, show="*")
        password_entry.grid(row=0, column=1, pady=10)

        # Action choice (Entrée/Sortie)
        ttk.Label(main_frame, text="Action:").grid(row=1, column=0, pady=10)
        action_frame = ttk.Frame(main_frame)
        action_frame.grid(row=1, column=1, pady=10)

        ttk.Radiobutton(action_frame, text="Entrée", variable=self.action_var,
                       value="Entrée").pack(side=tk.LEFT, padx=5)
        ttk.Radiobutton(action_frame, text="Sortie", variable=self.action_var,
                       value="Sortie").pack(side=tk.LEFT, padx=5)

        # Salle selection
        ttk.Label(main_frame, text="Numéro de salle:").grid(row=2, column=0, pady=10)
        salles = [str(num) for num in range(100, 111)]  # Crée une liste de "100" à "110"
        salle_combobox = ttk.Combobox(main_frame, textvariable=self.salle_var,
                                     values=salles, state="readonly", width=17)
        salle_combobox.grid(row=2, column=1, pady=10)

        # Login button
        ttk.Button(main_frame, text="Connexion",
                  command=self.verify_password).grid(row=3, column=0,
                                                   columnspan=2, pady=20)

        # Center the frame
        self.root.grid_rowconfigure(0, weight=1)
        self.root.grid_columnconfigure(0, weight=1)

    def verify_password(self):
        entered_password = self.password_var.get()
        action = self.action_var.get()
        salle = self.salle_var.get()

        # Hash the entered password
        hashed_password = hashlib.sha256(entered_password.encode()).hexdigest()

        try:
            # Load the stored password
            with open("password_config.json", "r") as f:
                config = json.load(f)
                stored_password = config.get("password")

            if hashed_password == stored_password:
                self.root.destroy()  # Close login window
                # Start main application
                root = tk.Tk()
                app = FaceObjectRecognitionApp(root, action, salle)
                root.mainloop()
            else:
                messagebox.showerror("Erreur", "Mot de passe incorrect")
                self.password_var.set("")  # Clear password field

        except FileNotFoundError:
            messagebox.showerror("Erreur", "Fichier de configuration non trouvé")

    def run(self):
        self.root.mainloop()

class FaceObjectRecognitionApp:
    def __init__(self, root, action="Entrée", salle="100"):
        self.root = root
        self.action = action  # Stocke l'action (Entrée/Sortie)
        self.salle = salle    # Stocke le numéro de salle
        self.root.title(f"Système de Reconnaissance - {action} - Salle {salle}")
        self.root.geometry("600x500")

        # Variables
        self.is_recognition_running = False
        self.training_in_progress = False
        self.model_yolo = None
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

        # Objets surveillés
        self.OBJETS_SURVEILLES = {
            'Sunglasses': 505,
            'Hat': 243,
            'Shorts': 456,
            'Skirt': 463,
            'Miniskirt': 334,
            'Helmet': 248,
            'Kitchen Knife': 292,
            'Shotgun': 457,
            'Handgun': 238
        }

        # Style
        style = ttk.Style()
        style.configure('Big.TButton', padding=10, font=('Helvetica', 12))

        # Création de l'interface
        self.create_widgets()
        self.last_logged_person = None  # Pour tracker la dernière personne enregistrée
        self.csv_file = "face_detection_logs.csv"

    def create_widgets(self):
        main_frame = ttk.Frame(self.root, padding="20")
        main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

        # Titre
        title_label = ttk.Label(main_frame,
                              text="Système de Reconnaissance Faciale et d'Objets",
                              font=('Helvetica', 14, 'bold'), wraplength=400)
        title_label.grid(row=0, column=0, pady=20)

        # Boutons
        register_btn = ttk.Button(main_frame, text="Enregistrer un nouveau visage",
                               command=self.show_register_dialog, style='Big.TButton')
        register_btn.grid(row=1, column=0, pady=10, padx=20, sticky=tk.EW)

        train_btn = ttk.Button(main_frame, text="Entraîner le modèle facial",
                             command=self.start_training, style='Big.TButton')
        train_btn.grid(row=2, column=0, pady=10, padx=20, sticky=tk.EW)

        load_yolo_btn = ttk.Button(main_frame, text="Charger le modèle YOLO",
                                 command=self.load_yolo_model, style='Big.TButton')
        load_yolo_btn.grid(row=3, column=0, pady=10, padx=20, sticky=tk.EW)

        recognize_btn = ttk.Button(main_frame, text="Lancer la reconnaissance",
                                 command=self.toggle_recognition, style='Big.TButton')
        recognize_btn.grid(row=4, column=0, pady=10, padx=20, sticky=tk.EW)

        analyze_img_btn = ttk.Button(main_frame, text="Analyser une image",
                                   command=self.analyze_image, style='Big.TButton')
        analyze_img_btn.grid(row=5, column=0, pady=10, padx=20, sticky=tk.EW)

        # Progress bar
        self.progress_var = tk.DoubleVar()
        self.progress = ttk.Progressbar(main_frame, variable=self.progress_var,
                                      maximum=100, mode='determinate')
        self.progress.grid(row=6, column=0, pady=10, sticky=tk.EW)

        # Status
        self.status_var = tk.StringVar(value="En attente...")
        status_label = ttk.Label(main_frame, textvariable=self.status_var,
                               wraplength=400)
        status_label.grid(row=7, column=0, pady=10)

    def show_register_dialog(self):
        """Affiche une fenêtre de dialogue pour enregistrer un nouveau visage"""
        dialog = tk.Toplevel(self.root)
        dialog.title("Enregistrer un nouveau visage")
        dialog.geometry("300x150")

        ttk.Label(dialog, text="Prénom:").grid(row=0, column=0, padx=5, pady=5)
        prenom_var = tk.StringVar()
        ttk.Entry(dialog, textvariable=prenom_var).grid(row=0, column=1, padx=5, pady=5)

        ttk.Label(dialog, text="Nom:").grid(row=1, column=0, padx=5, pady=5)
        nom_var = tk.StringVar()
        ttk.Entry(dialog, textvariable=nom_var).grid(row=1, column=1, padx=5, pady=5)

        def validate():
            prenom = prenom_var.get().strip()
            nom = nom_var.get().strip()
            if prenom and nom:
                dialog.destroy()
                self.capture_face(prenom, nom)
            else:
                messagebox.showerror("Erreur", "Veuillez remplir tous les champs")

        ttk.Button(dialog, text="Commencer la capture",
                  command=validate).grid(row=2, column=0, columnspan=2, pady=20)

    def capture_face(self, prenom, nom):
        """Capture les images du visage pour l'enregistrement"""
        # Créer le dossier pour enregistrer les images
        folder_name = f"dataset(test)/{prenom.capitalize()} {nom.capitalize()}"
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)

        # Ouvrir la caméra
        cap = cv2.VideoCapture(0)
        captured_images = 0
        directions = ["Regardez en face", "Regardez en haut", "Regardez vers la droite",
                     "Regardez en bas", "Regardez vers la gauche"]
        images_per_direction = 1
        total_images = len(directions) * images_per_direction

        direction_index = 0

        while captured_images < total_images:
            current_direction = directions[direction_index]

            # Pause de 3 secondes avec affichage
            for i in range(3, 0, -1):
                ret, frame = cap.read()
                if not ret:
                    break
                cv2.putText(frame, f"{current_direction} dans {i} secondes",
                           (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
                cv2.imshow('Capturer le visage', frame)
                cv2.waitKey(1000)

            while captured_images < (direction_index + 1) * images_per_direction:
                ret, frame = cap.read()
                if not ret:
                    break

                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                faces = self.face_cascade.detectMultiScale(gray, scaleFactor=1.1,
                                                         minNeighbors=5, minSize=(30, 30))

                for (x, y, w, h) in faces:
                    # Élargir la zone pour la tête
                    x_head = max(0, x - int(0.2 * w))
                    y_head = max(0, y - int(0.3 * h))
                    w_head = min(frame.shape[1] - x_head, int(w * 1.4))
                    h_head = min(frame.shape[0] - y_head, int(h * 1.6))

                    head = frame[y_head:y_head+h_head, x_head:x_head+w_head]
                    head_filename = f"{folder_name}/head_{captured_images}.jpg"
                    cv2.imwrite(head_filename, head)
                    captured_images += 1

                    cv2.putText(frame, f"Image {captured_images}/{total_images}",
                              (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    cv2.rectangle(frame, (x_head, y_head),
                                (x_head+w_head, y_head+h_head), (0, 255, 0), 2)

                cv2.putText(frame, f"Etape actuelle : {current_direction}",
                          (10, frame.shape[0] - 20),
                          cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

                cv2.imshow('Capturer le visage', frame)

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    cap.release()
                    cv2.destroyAllWindows()
                    return

            direction_index = (direction_index + 1) % len(directions)

        cap.release()
        cv2.destroyAllWindows()
        messagebox.showinfo("Succès",
                          "Enregistrement du visage terminé. Veuillez entraîner le modèle.")

    def load_models(self):
        """Charge les modèles nécessaires"""
        try:
            self.status_var.set("Chargement du modèle facial...")
            self.face_model = load_model("face_recognition_model.h5", compile=False)
            with open("label_map.pkl", 'rb') as f:
                self.label_map = pickle.load(f)
            self.status_var.set("Modèle facial chargé")
            return True
        except Exception as e:
            messagebox.showerror("Erreur", f"Erreur lors du chargement du modèle facial: {str(e)}")
            self.status_var.set("Erreur chargement modèle facial")
            return False

    def load_yolo_model(self):
        """Charge le modèle YOLO"""
        try:
            self.status_var.set("Chargement du modèle YOLO...")
            logging.getLogger("ultralytics").setLevel(logging.WARNING)
            self.model_yolo = YOLO("yolov8n-oiv7.pt", verbose=False)
            messagebox.showinfo("Succès", "Modèle YOLO chargé avec succès!")
            self.status_var.set("Modèle YOLO chargé")
        except Exception as e:
            messagebox.showerror("Erreur", f"Erreur lors du chargement de YOLO: {str(e)}")
            self.status_var.set("Erreur chargement YOLO")

    def detect_objects(self, frame):
        """Détecte les objets dans la frame"""
        if self.model_yolo is None:
            return []

        old_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

        results = self.model_yolo(frame, verbose=False)

        sys.stdout = old_stdout

        detections = []

        for r in results:
            boxes = r.boxes
            for box in boxes:
                cls_id = int(box.cls[0])
                for nom_objet, id_classe in self.OBJETS_SURVEILLES.items():
                    if cls_id == id_classe:
                        x1, y1, x2, y2 = box.xyxy[0]
                        confidence = float(box.conf[0])
                        if confidence > 0.4:
                            detections.append({
                                'objet': nom_objet,
                                'confiance': confidence,
                                'coords': (int(x1), int(y1), int(x2), int(y2))
                            })
        return detections

    def log_detection(self, personne, objets_interdits):
        """
        Enregistre une détection dans le fichier CSV si ce n'est pas un doublon consécutif
        """
        if personne == "Inconnu" or personne == self.last_logged_person:
            return False

        current_time = datetime.now()
        date = current_time.strftime('%d/%m/%Y')
        heure = current_time.strftime('%H:%M:%S')

        objets_str = "|".join(objets_interdits) if objets_interdits else "Aucun"

        with open(self.csv_file, 'a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f, delimiter=';')
            writer.writerow([date, heure, personne, objets_str, self.action, self.salle])

        self.last_logged_person = personne
        return True

    def start_recognition(self):
        """Fonction de reconnaissance en temps réel avec logging"""
        try:
            # Charger le modèle de reconnaissance faciale
            model = load_model("face_recognition_model.h5", compile=False)
            with open("label_map.pkl", 'rb') as f:
                label_map = pickle.load(f)

            cap = cv2.VideoCapture(0)
            face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

            last_predictions = []
            PREDICTION_MEMORY = 5

            while self.is_recognition_running:
                ret, frame = cap.read()
                if not ret:
                    break

                # Détection d'objets
                detections = self.detect_objects(frame)
                objets_interdits = []
                for detection in detections:
                    if detection['objet'] in self.OBJETS_SURVEILLES:
                        objets_interdits.append(detection['objet'])
                    x1, y1, x2, y2 = detection['coords']
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                    text = f"{detection['objet']} ({detection['confiance']:.2f})"
                    cv2.putText(frame, text, (x1, y1-10),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

                # Reconnaissance faciale
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

                for (x, y, w, h) in faces:
                    margin = 20
                    y1 = max(0, y - margin)
                    y2 = min(frame.shape[0], y + h + margin)
                    x1 = max(0, x - margin)
                    x2 = min(frame.shape[1], x + w + margin)
                    face_img = frame[y1:y2, x1:x2]

                    try:
                        embedding = DeepFace.represent(face_img, model_name="ArcFace",
                                                     enforce_detection=False)

                        if embedding:
                            embedding_array = np.array([embedding[0]["embedding"]])
                            prediction = model.predict(embedding_array, verbose=0)
                            predicted_class = np.argmax(prediction)
                            confidence = prediction[0][predicted_class]

                            # Système de lissage
                            last_predictions.append((predicted_class, confidence))
                            if len(last_predictions) > PREDICTION_MEMORY:
                                last_predictions.pop(0)

                            if len(last_predictions) >= 3:
                                class_counts = {}
                                for pred_class, conf in last_predictions:
                                    if pred_class not in class_counts:
                                        class_counts[pred_class] = []
                                    class_counts[pred_class].append(conf)

                                most_common_class = max(class_counts.items(),
                                                      key=lambda x: (len(x[1]),
                                                                   sum(x[1])/len(x[1])))[0]
                                avg_confidence = np.mean(class_counts[most_common_class])

                                predicted_class = most_common_class
                                confidence = avg_confidence

                            predicted_name = label_map[predicted_class]
                            if confidence < 0.75:  # Seuil de confiance
                                predicted_name = "Inconnu"
                                color = (0, 0, 255)  # Rouge pour inconnu
                                text = predicted_name
                            else:
                                predicted_name = label_map[predicted_class]
                                color = (0, int(255 * confidence), 0)
                                text = f"{predicted_name} ({confidence:.2f})"
                                self.log_detection(predicted_name, objets_interdits)

                            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
                            cv2.putText(frame, text, (x, y-10),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                    except Exception as e:
                        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)

                cv2.imshow('Système de reconnaissance', frame)

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

            cap.release()
            cv2.destroyAllWindows()

        except Exception as e:
            messagebox.showerror("Erreur", f"Erreur pendant la reconnaissance: {str(e)}")
            self.status_var.set("Erreur pendant la reconnaissance")
        finally:
            self.is_recognition_running = False

    def analyze_image(self):
        """Analyse une image sélectionnée"""
        # Ouvrir le sélecteur de fichier
        file_path = filedialog.askopenfilename(
            title="Sélectionner une image",
            filetypes=[("Images", "*.jpg *.jpeg *.png *.bmp")]
        )

        if not file_path:
            return

        # Vérifier que les modèles sont chargés
        if not self.load_models():
            return

        try:
            # Lire l'image
            frame = cv2.imread(file_path)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convertir en RGB pour DeepFace

            # Détection d'objets
            detections = self.detect_objects(frame)
            for detection in detections:
                x1, y1, x2, y2 = detection['coords']
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                text = f"{detection['objet']} ({detection['confiance']:.2f})"
                cv2.putText(frame, text, (x1, y1-10),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

            # Reconnaissance faciale
            face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +
                                               'haarcascade_frontalface_default.xml')
            gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

            for (x, y, w, h) in faces:
                margin = 20
                y1 = max(0, y - margin)
                y2 = min(frame.shape[0], y + h + margin)
                x1 = max(0, x - margin)
                x2 = min(frame.shape[1], x + w + margin)
                face_img = frame[y1:y2, x1:x2]

                try:
                    embedding = DeepFace.represent(face_img, model_name="ArcFace",
                                                 enforce_detection=False)

                    if embedding:
                        embedding_array = np.array([embedding[0]["embedding"]])
                        prediction = self.face_model.predict(embedding_array, verbose=0)
                        predicted_class = np.argmax(prediction)
                        confidence = prediction[0][predicted_class]

                        predicted_name = self.label_map[predicted_class]
                        if confidence < 0.75:  # Seuil de confiance
                            predicted_name = "Inconnu"
                            color = (0, 0, 255)  # Rouge pour inconnu
                            text = predicted_name
                        else:
                            predicted_name = self.label_map[predicted_class]
                            color = (0, int(255 * confidence), 0)
                            text = f"{predicted_name} ({confidence:.2f})"

                        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)

                        cv2.putText(frame, text, (x, y-10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                except Exception as e:
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)

            # Afficher l'image analysée
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)  # Reconvertir en BGR pour l'affichage
            cv2.imshow('Analyse de l\'image', frame)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

        except Exception as e:
            messagebox.showerror("Erreur", f"Erreur pendant l'analyse de l'image: {str(e)}")
            self.status_var.set("Erreur pendant l'analyse")

    def toggle_recognition(self):
        """Démarre ou arrête la reconnaissance"""
        if not self.is_recognition_running:
            if not os.path.exists("face_recognition_model.h5"):
                messagebox.showerror("Erreur", "Le modèle facial n'a pas été entraîné!")
                return

            if self.model_yolo is None:
                messagebox.showwarning("Attention",
                                     "Le modèle YOLO n'est pas chargé. " +
                                     "Seule la reconnaissance faciale sera active.")

            self.is_recognition_running = True
            self.status_var.set("Reconnaissance en cours...")
            threading.Thread(target=self.start_recognition, daemon=True).start()
        else:
            self.is_recognition_running = False
            self.status_var.set("Reconnaissance arrêtée")

    def train_model(self):
        """Fonction d'entraînement du modèle"""
        try:
            self.status_var.set("Chargement des données...")
            db_path = "dataset(test)"
            embeddings = []
            labels = []
            label_map = {}

            # Liste tous les dossiers de personnes
            all_persons = sorted([d for d in os.listdir(db_path)
                                if os.path.isdir(os.path.join(db_path, d))])
            total_persons = len(all_persons)

            for idx, person in enumerate(all_persons):
                person_path = os.path.join(db_path, person)
                label_map[idx] = person

                person_progress = (idx / total_persons) * 50  # Première moitié de la progress bar
                self.progress_var.set(person_progress)
                self.status_var.set(f"Traitement des images de {person}...")

                for image_name in os.listdir(person_path):
                    image_path = os.path.join(person_path, image_name)
                    try:
                        embedding = DeepFace.represent(image_path, model_name="ArcFace",
                                                     enforce_detection=False)
                        embeddings.append(embedding[0]["embedding"])
                        labels.append(idx)
                    except Exception as e:
                        print(f"Erreur avec {image_path}: {e}")

            embeddings = np.array(embeddings)
            labels = np.array(labels)

            # Préparation des données
            self.status_var.set("Préparation des données...")
            X_train, X_test, y_train, y_test = train_test_split(embeddings, labels,
                                                               test_size=0.2, random_state=42)
            y_train = to_categorical(y_train, num_classes=len(label_map))
            y_test = to_categorical(y_test, num_classes=len(label_map))

            # Création et compilation du modèle
            self.status_var.set("Création du modèle...")
            model = Sequential([
                Dense(128, input_shape=(embeddings.shape[1],), activation='relu'),
                Dropout(0.5),
                Dense(64, activation='relu'),
                Dropout(0.5),
                Dense(len(label_map), activation='softmax')
            ])

            model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

            # Entraînement
            self.status_var.set("Entraînement en cours...")
            epochs = 100
            for epoch in range(epochs):
                model.fit(X_train, y_train, epochs=1, batch_size=16,
                         validation_data=(X_test, y_test), verbose=0)
                progress = 50 + (epoch / epochs) * 50  # Deuxième moitié de la progress bar
                self.progress_var.set(progress)
                self.status_var.set(f"Entraînement: {epoch+1}/{epochs} epochs")

            # Sauvegarde
            model.save("face_recognition_model.h5")
            with open("label_map.pkl", 'wb') as f:
                pickle.dump(label_map, f)

            self.progress_var.set(100)
            self.status_var.set("Entraînement terminé!")
            messagebox.showinfo("Succès", "Le modèle a été entraîné avec succès!")

        except Exception as e:
            self.status_var.set("Erreur pendant l'entraînement")
            messagebox.showerror("Erreur", f"Erreur pendant l'entraînement: {str(e)}")
        finally:
            self.training_in_progress = False
            self.progress_var.set(0)

    def start_training(self):
            """Démarre l'entraînement dans un thread séparé"""
            if not self.training_in_progress:
                self.training_in_progress = True
                threading.Thread(target=self.train_model, daemon=True).start()
            else:
                messagebox.showwarning("En cours", "L'entraînement est déjà en cours!")

if __name__ == "__main__":
    login = LoginWindow()
    login.run()